In [1]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
c = Client(cluster)

c


/Users/auraoupa/anaconda2/envs/gonzag/lib/python3.9/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55509 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:55510 Dashboard: http://127.0.0.1:55509/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [2]:
import gonzag as gz
from gonzag.config import ldebug


In [7]:
file_sat = 'zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika_chunksby1000'
name_ssh_sat = 'adt_unfiltered'
file_mod = 'zarr_ssh_ORCA1_20170101_20171231_grid_T_chunksby100'
name_ssh_mod = 'ssh'
file_lsm_mod = 'zarr_ssh_ORCA1_20170101_20171231_grid_T_chunksby100'
name_lsm_mod = '_FillValue'

In [8]:
(it1,it2), (Nts,Ntm) = gz.GetEpochTimeOverlap( file_sat , file_mod )

import zarr
 *** [GetTimeInfo()] Getting calendar/time info in zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika_chunksby1000 ...
   => time/record dimension is "time"
 *** [GetTimeInfo()] Getting calendar/time info in zarr_ssh_ORCA1_20170101_20171231_grid_T_chunksby100 ...
   => time/record dimension is "time_counter"

 *** [GetEpochTimeOverlap()] Earliest/latest dates:
   => for satellite data: 2017-04-01T23:57:40.480926208 2017-04-02T23:14:42.618708992 
   => for model     data: 2017-01-16T12:00:00.000000000 2017-12-16T12:00:00.000000000 



In [9]:
    print(' *** Time overlap between model and satellite in UNIX epoch time: it1, it2', it1,'--',it2)
    print('   => UTC: "'+str(it1)+'" to "'+str(it2)+'"\n')


    print('\n\n\n #####   M O D E L   2 D + T   D O M A I N   a.k.a.  S O U R C E   #####\n')



 *** Time overlap between model and satellite in UNIX epoch time: it1, it2 2017-04-01T23:57:40.480926208 -- 2017-04-02T23:14:42.618708992
   => UTC: "2017-04-01T23:57:40.480926208" to "2017-04-02T23:14:42.618708992"




 #####   M O D E L   2 D + T   D O M A I N   a.k.a.  S O U R C E   #####



In [10]:
clsm = name_lsm_mod
if name_lsm_mod=='_FillValue': clsm = name_lsm_mod+'@'+name_ssh_mod

In [11]:
ModelGrid = gz.ModGrid( file_mod, it1, it2, file_lsm_mod, clsm )

 *** [GetTimeEpochVector()] reading "time_counter" in zarr_ssh_ORCA1_20170101_20171231_grid_T_chunksby100 and converting it to Epoch time...
 *** [GetTimeEpochVector()] reading "time_counter" in zarr_ssh_ORCA1_20170101_20171231_grid_T_chunksby100 and converting it to Epoch time...
 *** Model latitude var is: "nav_lat" ! with 2 dimensions! (292, 362) 

 *** Model longitude var is: "nav_lon" ! with 2 dimensions! (292, 362) 


 *** what we use to define model land-sea mask:
    => "_FillValue@ssh" in "zarr_ssh_ORCA1_20170101_20171231_grid_T_chunksby100"

 *** [GridResolution()] Based on the longitude array, the model resolution ~=  1.0  degrees 

 *** Computing angle distortion of the model grid...

 *** About model gridded (source) domain:
     * shape =  (292, 362)
     * horizontal resolution:  1.0  degrees or  111.11  km
     * lon_min, lon_max =  0.0 360.0
     * Is this a global domain w.r.t longitude:  True
       ==> East West periodicity:  True , with an overlap of  2  points
   

In [12]:
    print('\n\n\n #####   S A T E L L I T E   1 D   T R A C K   a.k.a.  T A R G E T   #####\n')

    SatelliteTrack = gz.SatTrack( file_sat, it1, it2, Np=Nts, \
                                  domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )





 #####   S A T E L L I T E   1 D   T R A C K   a.k.a.  T A R G E T   #####

 *** [SatTrack()] Analyzing the time vector in zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika_chunksby1000 ...
 *** [GetTimeEpochVector()] reading "time" in zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika_chunksby1000 and converting it to Epoch time...
 *** [GetSatCoord()] reading "latitude" in zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika_chunksby1000 ...
   => 44533 records in TOTAL!

 *** [GetSatCoord()] reading "longitude" in zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika_chunksby1000 ...
   => 44533 records in TOTAL!


 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  44533
       ==> time record indices: 0 to 44532, included



In [13]:
MG= ModelGrid
ST=SatelliteTrack
file_out='result.nc'

In [14]:
from gonzag.zarrio import GetModel2DVar, Save2Dfield, GetSatSSH, SaveTimeSeries

In [15]:
import time ; # to report execution speed of certain parts of the code...
#
from gonzag.config import IsZarr, ldebug, if_talk, l_plot_meshes, deg2km, rfactor, search_box_w_km, l_save_track_on_model_grid, l_plot_meshes, rmissval
from gonzag.utils  import *
from gonzag.bilin_mapping import BilinTrack


In [16]:
    (Nj,Ni) = MG.shape

    d_found_km = rfactor*MG.HResDeg*deg2km
    print(' *** "found" distance criterion when searching for nearest point on model grid is ', d_found_km, ' km\n')

    # Size of the search zoom box:
    np_box_radius = SearchBoxSize( MG.HResDeg*deg2km, search_box_w_km )
    print(' *** Will use zoom boxes of width of '+str(2*np_box_radius+1)+' points for 1st attempts of nearest-point location...\n')

    Nt = ST.size ; # number of satellit observation point to work with here...


    # The BIG GUY:
    startTime = time.time()



 *** "found" distance criterion when searching for nearest point on model grid is  83.3325  km

 *** Will use zoom boxes of width of 5 points for 1st attempts of nearest-point location...



In [17]:
%time     BT = BilinTrack( ST.lat, ST.lon, MG.lat, MG.lon, src_grid_local_angle=MG.xangle, \
                     k_ew_per=MG.EWPer, rd_found_km=d_found_km, np_box_r=np_box_radius, freq_talk=if_talk )
 


 *** Finding nearest points on source (model) grid... (rd_found_km, np_box_r = 83.3325 2 )
      +++ Treated point: 500/44533 
          ==> Sat. coordinates:     -34.85 94.606
          ==> Model nearest point:  -34.668 94.5  ( 94 22 )
      +++ Treated point: 1000/44533 
          ==> Sat. coordinates:     -3.741 87.002
          ==> Model nearest point:  -3.694 87.5  ( 135 15 )
      +++ Treated point: 1500/44533 
          ==> Sat. coordinates:     11.411 256.152
          ==> Model nearest point:  11.385 256.5  ( 172 184 )
      +++ Treated point: 2000/44533 
          ==> Sat. coordinates:     -19.618 249.192
          ==> Model nearest point:  -19.606 249.5  ( 111 177 )
      +++ Treated point: 2500/44533 
          ==> Sat. coordinates:     -50.445 239.635
          ==> Model nearest point:  -50.697 239.5  ( 72 167 )
      +++ Treated point: 3000/44533 
          ==> Sat. coordinates:     -59.547 79.973
          ==> Model nearest point:  -59.771 79.5  ( 56 7 )
      +++ Treat

      +++ Treated point: 30000/44533 
          ==> Sat. coordinates:     34.257 35.487
          ==> Model nearest point:  34.037 35.571  ( 198 323 )
      +++ Treated point: 30500/44533 
          ==> Sat. coordinates:     -64.315 4.554
          ==> Model nearest point:  -64.439 4.5  ( 46 292 )
      +++ Treated point: 31000/44533 
          ==> Sat. coordinates:     -49.213 208.093
          ==> Model nearest point:  -49.413 208.5  ( 74 136 )
      +++ Treated point: 31500/44533 
          ==> Sat. coordinates:     -18.245 198.745
          ==> Model nearest point:  -18.662 198.5  ( 112 126 )
      +++ Treated point: 32000/44533 
          ==> Sat. coordinates:     15.433 191.192
          ==> Model nearest point:  15.824 191.5  ( 177 119 )
      +++ Treated point: 32500/44533 
          ==> Sat. coordinates:     46.014 182.489
          ==> Model nearest point:  46.334 182.695  ( 213 110 )
      +++ Treated point: 33000/44533 
          ==> Sat. coordinates:     39.324 11.882
    

 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is t

WARNING / sosie_bilin.Iquadran(): point not inside polygon!
 * Model: jP, iP = 205 288  GPS =>  39.96417999267578 0.7227873802185059
          local distortion of the grid => -2.053308230926231  degrees
 * Sat:  yT, zT = 39.947053 0.714313
Headings:
 * direction target:      ht = 200.77086257679213     hz = -159.22913742320787
 * direction point above: hN = 2.168349144101955
 * direction point rhs:   hE = 92.08217490331423
 * direction point below: hS = 181.94175745399758
 * direction point lhs:   hW = 272.02276557662947
   iquadran to be used = 3 

WARNING / sosie_bilin.Iquadran(): point not inside polygon!
 * Model: jP, iP = 205 288  GPS =>  39.96417999267578 0.7227873802185059
          local distortion of the grid => -2.053308230926231  degrees
 * Sat:  yT, zT = 40.007914 0.6943809999999999
Headings:
 * direction target:      ht = 333.54196213859086     hz = -26.45803786140914
 * direction point above: hN = 2.168349144101955
 * direction point rhs:   hE = 92.08217490331423
 * direc

 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is t

...

 * direction point lhs:   hW = 270.0
   iquadran to be used = 4 

WARNING / sosie_bilin.Iquadran(): point not inside polygon!
 * Model: jP, iP = 125 288  GPS =>  -7.917142868041992 0.5
          local distortion of the grid => 0.0  degrees
 * Sat:  yT, zT = -7.900128 0.338997
Headings:
 * direction target:      ht = 276.0901473158748     hz = -83.90985268412521
 * direction point above: hN = 0.0
 * direction point rhs:   hE = 90.0
 * direction point below: hS = 180.0
 * direction point lhs:   hW = 270.0
   iquadran to be used = 4 

WARNING / sosie_bilin.Iquadran(): point not inside polygon!
 * Model: jP, iP = 125 288  GPS =>  -7.917142868041992 0.5
          local distortion of the grid => 0.0  degrees
 * Sat:  yT, zT = -7.961612 0.32531
Headings:
 * direction target:      ht = 255.58562001920717     hz = -104.41437998079283
 * direction point above: hN = 0.0
 * direction point rhs:   hE = 90.0
 * direction point below: hS = 180.0
 * direction point lhs:   hW = 270.0
   iquadran to be 

 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is t

 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
 *** / sosie_bilin.Iquadran(): need to go for heavy-duty mode in search for "iquadran"!
      ==> because first attempt did not succeed
 ===> FOUND! iq= 4  is the guy!
WARNING / sosie_bilin.Iquadran(): point not inside polygon!
 * Model: jP, iP = 267 275  GPS =>  77.97159576416016 2.686331272125244
          local distortion of the grid => -21.290365751203517  degrees
 * Sat:  yT, zT = 77.972495 1.978298
Headings:
 * direction target:      ht = 270.34919064905773     hz = -89.65080935094227


/Users/auraoupa/Work/git/gonzag/gonzag/bilin_mapping.py:108: RuntimeWarning: invalid value encountered in double_scalars
  zdalp = nmp.linalg.det( zM ) / zdeta
/Users/auraoupa/Work/git/gonzag/gonzag/bilin_mapping.py:111: RuntimeWarning: invalid value encountered in double_scalars
  zdbet = nmp.linalg.det( zM ) / zdeta


   WARNING / sosie_bilin.WeightBL(): will use nearest point value
             as source mesh could not be identidied!
    * Model:jP, iP = -1 -1  GPS =>  50.01094055175781 72.98915100097656
    * Sat:  yT, xT = 72.995846 1.385794
   WARNING / sosie_bilin.WeightBL(): will use nearest point value
             as source mesh could not be identidied!
    * Model:jP, iP = -1 -1  GPS =>  50.01094055175781 72.98915100097656
    * Sat:  yT, xT = 65.659751 0.888747
   WARNING / sosie_bilin.WeightBL(): will use nearest point value
             as source mesh could not be identidied!
    * Model:jP, iP = -1 -1  GPS =>  50.01094055175781 72.98915100097656
    * Sat:  yT, xT = 65.60171199999999 0.83044
   WARNING / sosie_bilin.WeightBL(): will use nearest point value
             as source mesh could not be identidied!
    * Model:jP, iP = -1 -1  GPS =>  50.01094055175781 72.98915100097656
    * Sat:  yT, xT = 65.543655 0.772372
     ***    Done! *** 

CPU times: user 12.6 s, sys: 1.41 s, total: 1

In [18]:
    time_bl_mapping = time.time() - startTime



In [19]:
    vssh_m_np = nmp.zeros(Nt) ; vssh_m_np[:] = rmissval; # vector to store the model data interpolated in time and space (nearest-point) on the satellite track...
    vssh_m_bl = nmp.zeros(Nt) ; vssh_m_bl[:] = rmissval; # vector to store the model data interpolated in time and space (bilinear) on the satellite track...
    vdistance = nmp.zeros(Nt)

    # Time increment on the satellite time:
    ktm1   = 0   ; ktm2   = 0
    ktm1_o = -10 ; ktm2_o = -10

    startTime = time.time()


In [20]:
print(Nt)

44533


In [21]:
jt=0

In [24]:
%%time
        itt = ST.time[jt] ; # unix time

        # Get surrounding records for model:
        kt = ktm1
        while not (MG.time[kt]<=itt and MG.time[kt+1]>itt): kt=kt+1
        ktm1 = kt ; ktm2 = kt+1



CPU times: user 35 µs, sys: 1 µs, total: 36 µs
Wall time: 38.9 µs


In [26]:
%%time
        if jt%if_talk==0:
            print('\n *** Satelite time at jt = '+'%5.5i'%(jt)+' ==> ',EpochT2Str(itt), itt)
            print('   => surounding kt for model: ', ktm1, ktm2, '(',EpochT2Str(MG.time[ktm1]),EpochT2Str(MG.time[ktm2]),') / ', \
                  MG.time[ktm1],MG.time[ktm2] )





 *** Satelite time at jt = 00000 ==>  2017-04-01T23:57:40.480926208 2017-04-01T23:57:40.480926208
   => surounding kt for model:  0 1 ( 2017-03-16T12:00:00.000000000 2017-04-16T00:00:00.000000000 ) /  2017-03-16T12:00:00.000000000 2017-04-16T00:00:00.000000000
CPU times: user 352 µs, sys: 218 µs, total: 570 µs
Wall time: 409 µs


In [29]:
%%time
        # If first time we have these ktm1 & ktm2, getting the two surrounding fields:
        if (ktm1>ktm1_o) and (ktm2>ktm2_o):
            if (ktm1_o == -10) or (ktm1 > ktm2_o):
                print(' *** Reading '+name_ssh_mod+' in '+MG.file+'\n    => at ktm1=', ktm1)
                Xm1 = GetModel2DVar( MG.file, name_ssh_mod, kt=ktm1 )
            else:
                Xm1[:,:] = Xm2[:,:]
            #
            print(' *** Reading '+name_ssh_mod+' in '+MG.file+'\n    => at ktm2=', ktm2)
            Xm2 = GetModel2DVar( MG.file, name_ssh_mod, kt=ktm2 )

            # slope only needs to be calculated when Xm2 and Xm1 have been updated:
            Xa = (Xm2 - Xm1) / float(MG.time[ktm2] - MG.time[ktm1])



CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


In [39]:
%%time
        # Linear interpolation of field at time itt:
        Xm = Xm1[:,:] + Xa[:,:]*float(itt - MG.time[ktm1])



CPU times: user 46.9 ms, sys: 6.23 ms, total: 53.1 ms
Wall time: 88.8 ms


In [33]:
%%time
        [ [j1,i1],[j2,i2],[j3,i3],[j4,i4] ] = BT.SM[jt,:,:]
        [w1, w2, w3, w4]                    = BT.WB[jt,:]



CPU times: user 23 µs, sys: 1 µs, total: 24 µs
Wall time: 24.8 µs


In [36]:
%%time
        Sm = MG.mask[j1,i1] + MG.mask[j2,i2] + MG.mask[j3,i3] + MG.mask[j4,i4]



CPU times: user 121 ms, sys: 6.49 ms, total: 128 ms
Wall time: 274 ms


In [40]:
%%time
        if Sm == 4:
            # All 4 model points are ocean point !

            vssh_m_np[jt] = Xm[j1,i1] ; # Nearest-point "interpolation"



CPU times: user 38 ms, sys: 4.84 ms, total: 42.9 ms
Wall time: 87.5 ms


In [41]:
%%time
        if Sm == 4:
            # Bilinear interpolation:
            Sw = nmp.sum([w1, w2, w3, w4])


CPU times: user 14.4 ms, sys: 2.34 ms, total: 16.7 ms
Wall time: 37.6 ms


In [42]:
%%time
        if Sm == 4:
            if abs(Sw-1.)> 0.001:
                print('    FLAGGING MISSING VALUE at jt = '+str(jt)+' !!!')
            else:
                vssh_m_bl[jt] = w1*Xm[j1,i1] + w2*Xm[j2,i2] + w3*Xm[j3,i3] + w4*Xm[j4,i4]


CPU times: user 38.2 ms, sys: 3.33 ms, total: 41.5 ms
Wall time: 95 ms


In [38]:
%%time
        ktm1_o = ktm1 ; ktm2_o = ktm2


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 18.1 µs


In [23]:
print(Nt*1.14/3660)

13.870934426229507
